GitHub repository: [github.com/gonzafernan/ceia-iia](https://github.com/gonzafernan/ceia-iia)

## Busquedas basadas en profundidad

Herramienta para benchmark de memoria utilizada por los algoritmos y calidad de camino encontrado:

In [5]:
import statistics
import tracemalloc
from typing import Callable

def memory_benchmark(fn: Callable, runs: int, **kwargs) -> tuple[float, float]:
    memory_peak_lst = []
    for _ in range(runs):
        tracemalloc.start()
        fn(**kwargs)
        _, memory_peak = tracemalloc.get_traced_memory()
        memory_peak /= 1024*1024
        tracemalloc.stop()
        memory_peak_lst.append(memory_peak)
    return statistics.mean(memory_peak_lst), statistics.stdev(memory_peak_lst)


def solution_path_benchmark(fn: Callable, runs: int, **kwargs) -> tuple[int, int]:
    path_solution = []
    for _ in range(runs):
        path_solution.append(fn(**kwargs).state.accumulated_cost)
    return statistics.mean(path_solution), statistics.stdev(path_solution)

Implementacion de los algoritmos:

In [6]:
from aima import Node, Problem
from typing import Optional

def depth_first_search(initial_node: Node, problem: Problem) -> Optional[Node]:
    """Depth-first search (DFS)"""
    frontier = [initial_node]
    explored = set()
    while len(frontier) > 0:
        node = frontier.pop()
        explored.add(node.state)
        if problem.goal_test(node.state):
            return node
        frontier.extend(list(filter(lambda x: not x.state in explored, node.expand(problem))))
    return None

def depth_limited_search(initial_node: Node, problem: Problem, depth: int, explored: set) -> tuple[Optional[Node], bool]:
    """Depth-limited search (DLS). Recursive implementation."""
    if depth == 0:
        return (initial_node, True) if problem.goal_test(initial_node.state) else (None, True)
    any_remaining = False
    for next_node in filter(lambda x: not x.state in explored, initial_node.expand(problem)):
        explored.add(next_node.state)
        ans_node, remaining = depth_limited_search(initial_node=next_node, problem=problem, depth=depth-1, explored=explored)
        if ans_node:
            return ans_node, True
        if remaining:
            any_remaining = True 
    return None, any_remaining


def iterative_deepening_depth_first_search(initial_node: Node, problem: Problem, max_depth: int) -> Optional[Node]:
    """Iterative deepening depth-first search (IDDFS)"""
    for depth in range(max_depth):
        ans_node, remaining = depth_limited_search(initial_node=initial_node, problem=problem, depth=depth, explored=set([initial_node.state]))
        if ans_node:
            return ans_node
        if not remaining:
            return None
    return None

Definicion del problema:

In [7]:
from hanoi_states import StatesHanoi, ProblemHanoi
from tree_hanoi import NodeHanoi

initial_state = StatesHanoi(rod1=[5, 4, 3, 2, 1], rod2=[], rod3=[], max_disks=5)
goal_state = StatesHanoi(rod1=[], rod2=[], rod3=[5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

Ejecucion de los algoritmos con analisis de tiempo y memoria:

In [8]:
%%timeit
dfs_result = depth_first_search(initial_node=NodeHanoi(problem.initial), problem=problem)

4.03 ms ± 63 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
memory_peak_mean, memory_peak_std = memory_benchmark(fn=depth_first_search, runs=20, initial_node=NodeHanoi(problem.initial), problem=problem)
print(f"Maxima memoria ocupada: {memory_peak_mean:.4f} [MB] +- {memory_peak_std:.4f}")

Maxima memoria ocupada: 0.2393 [MB] +- 0.0019


In [10]:
path_mean, path_std = solution_path_benchmark(fn=depth_first_search, runs=20, initial_node=NodeHanoi(problem.initial), problem=problem)
print(f"Longitud de camino: {path_mean} +- {path_std}")

Longitud de camino: 121.0 +- 0.0


In [11]:
%%timeit
iddfs_result = iterative_deepening_depth_first_search(initial_node=NodeHanoi(problem.initial), problem=problem, max_depth=150)

170 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
memory_peak_mean, memory_peak_std = memory_benchmark(iterative_deepening_depth_first_search, runs=20, initial_node=NodeHanoi(problem.initial), problem=problem, max_depth=150)
print(f"Maxima memoria ocupada: {memory_peak_mean:.4f} [MB] +- {memory_peak_std:.4f}")

Maxima memoria ocupada: 0.4189 [MB] +- 0.0476


In [13]:
path_mean, path_std = solution_path_benchmark(fn=iterative_deepening_depth_first_search, runs=20, initial_node=NodeHanoi(problem.initial), problem=problem, max_depth=150)
print(f"Longitud de camino: {path_mean} +- {path_std}")

Longitud de camino: 65.0 +- 0.0


In [14]:
iddfs_result = iterative_deepening_depth_first_search(initial_node=NodeHanoi(problem.initial), problem=problem, max_depth=150)
iddfs_result.generate_solution_for_simulator()